In [208]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import os
import time
import numpy as np

print(pd.__version__)

1.3.4


In [283]:
path = "../data"

df = pd.read_csv(path + "/netflix_titles.csv")

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,s1,Movie,Dick Johnson Is Dead,Kirsten Johnson,NaN,United States,"September 25, 2021",2020,PG-13,90 min,Documentaries,"As her father nears the end of his life, filmm..."
1,s2,TV Show,Blood & Water,NaN,"Ama Qamata, Khosi Ngema, Gail Mabalane, Thaban...",South Africa,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, TV Dramas, TV Mysteries","After crossing paths at a party, a Cape Town t..."
2,s3,TV Show,Ganglands,Julien Leclercq,"Sami Bouajila, Tracy Gotoas, Samuel Jouy, Nabi...",NaN,"September 24, 2021",2021,TV-MA,1 Season,"Crime TV Shows, International TV Shows, TV Act...",To protect his family from a powerful drug lor...
3,s4,TV Show,Jailbirds New Orleans,NaN,NaN,NaN,"September 24, 2021",2021,TV-MA,1 Season,"Docuseries, Reality TV","Feuds, flirtations and toilet talk go down amo..."
4,s5,TV Show,Kota Factory,NaN,"Mayur More, Jitendra Kumar, Ranjan Raj, Alam K...",India,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, Romantic TV Shows, TV ...",In a city of coaching centers known to train I...
5,s6,TV Show,Midnight Mass,Mike Flanagan,"Kate Siegel, Zach Gilford, Hamish Linklater, H...",NaN,"September 24, 2021",2021,TV-MA,1 Season,"TV Dramas, TV Horror, TV Mysteries",The arrival of a charismatic young priest brin...
6,s7,Movie,My Little Pony: A New Generation,"Robert Cullen, José Luis Ucha","Vanessa Hudgens, Kimiko Glenn, James Marsden, ...",NaN,"September 24, 2021",2021,PG,91 min,Children & Family Movies,Equestria's divided. But a bright-eyed hero be...
7,s8,Movie,Sankofa,Haile Gerima,"Kofi Ghanaba, Oyafunmike Ogunlano, Alexandra D...","United States, Ghana, Burkina Faso, United Kin...","September 24, 2021",1993,TV-MA,125 min,"Dramas, Independent Movies, International Movies","On a photo shoot in Ghana, an American model s..."
8,s9,TV Show,The Great British Baking Show,Andy Devonshire,"Mel Giedroyc, Sue Perkins, Mary Berry, Paul Ho...",United Kingdom,"September 24, 2021",2021,TV-14,9 Seasons,"British TV Shows, Reality TV",A talented batch of amateur bakers face off in...
9,s10,Movie,The Starling,Theodore Melfi,"Melissa McCarthy, Chris O'Dowd, Kevin Kline, T...",United States,"September 24, 2021",2021,PG-13,104 min,"Comedies, Dramas",A woman adjusting to life after a loss contend...


In [4]:
df.isnull().sum()

show_id            0
type               0
title              0
director        2634
cast             825
country          831
date_added        10
release_year       0
rating             4
duration           3
listed_in          0
description        0
dtype: int64

In [382]:
# 특수문자 및 공백 처리
for_uri = {
            " ": "+", "!": "%21", "\"": "%22", "#": "%23",
           "%": "%25", "&": "%26", "(": "%28", ")": "%29",
           "+": "%2B", "\\": "%5C", ",": "%2C", ".": "%2E",
           "/": "%2F", ":": "%3A", ";": "%3B", "<": "%3C",
           ">": "%3E", "=": "%3D", "?": "%3F", "@": "%40",
           "{": "%7B", "}": "%7D", "[": "%5B", "]": "%5D",
           "|": "%7C", "~": "%7E", "\'": "%27",
          }

In [415]:
# REQUESTS를 사용해 URL가져오기
def get_html(url):
    
    """
    
    Params: 홈페이지 URI 
    requests 라이브러리에서 .get 함수를 사용해 URI의 정보를 크롤링합니다. 
    Return: URI의 정보
    
    
    """
    
    _html = url
    
    headers = {
       "user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/99.0.4844.51 Safari/537.36"
    }
    
    resp = requests.get(url, headers = headers)
    if resp.status_code == 200:
        _html = resp.text
    return _html

# 특수문자 URI에 맞게
def to_uri(keyword):
    
    """
    
    Params: 검색 할 키워드
    Keyword를 URI 패턴에 맞게 공백 및 특수문자를 replace 합니다.
    Return: replace된 키워드
    
    """
    
    new = ""
    for i in keyword:
        try:
            if for_uri[i]:
                new += for_uri[i]

        except KeyError:
            new += i
        
    return new

# IMDB홈페이지의 title id 값 찾기
def find_title_id(keyword):
    
    """
    
    Params: 영화 및 드라마 title
    "https://www.imdb.com/?ref_=nv_home"의 영화 및 드라마 정보를 찾기 위해, 각 title에 대한 IMDB에서 사용한 고유ID를 찾는 함수
    Return: title_id(고유ID)
    
    """
    
    keyword = to_uri(keyword)
    URL = f"https://www.imdb.com/find?s=tt&q={keyword}&ref_=nv_sr_sm"
    resq = get_html(URL)
    soup = BeautifulSoup(resq, "html.parser")
    
    try:
        title_id =  soup.select("#main > div > div.findSection > table > tr:nth-child(1) > td.result_text > a")[0]["href"].split("/")[2]
        
    except IndexError:
        title_id = "NOT FOUND"
        
        
    print("Title :", keyword)
#     print("ID :", title_id)
    
    return title_id
    
# director, cast, cnty 결측값 채우기
def fill_dir_cast_cnty(keyword, cast_n = 10):
    
    """
    
    Params: 영화 및 드라마 title, 정보를 가지고 올 배우(cast) 수 
    title_id를 받아 "https://www.imdb.com/?ref_=nv_home"에서 제공하는 director, cast, country 정보를 크롤링 하는 함수
    Return: 크롤링한 director, cast, country 정보
    
    """
    
    director = []
    cast = []
    cnty = []
    
    title_id = find_title_id(keyword)
    if title_id == "NOT FOUND":
        director += "NOT FOUND"
        cast += "NOT FOUND"
        cnty += "NOT FOUND"
        
        
        return director, cast, cnty, title_id
    
    #######################################country##############################################
    
    URL = f"https://www.imdb.com/title/{title_id}/?ref_=fn_tt_tt_1"
    resq = get_html(URL)
    soup = BeautifulSoup(resq, "html.parser")
    
#     print("title_id :" ,title_id)
    try:
        country = soup.select_one('#__next > main > div > section.ipc-page-background.ipc-page-background--base.MainDetailPageLayout__StyledPageBackground-sc-13rp3wh-0.hsughJ > div > section > div > div.BTFWrapper__MainColumnContent-sc-1g8jgrg-1.fKIzsM.ipc-page-grid__item.ipc-page-grid__item--span-2 > section[data-testid="Details"] > div.styles__MetaDataContainer-sc-12uhu9s-0.cgqHBf > ul > li:nth-child(2) > div > ul > li > a')
        cnty.append(country.text.strip())
    except AttributeError:
        cnty.append("NONE")
        
    ######################################director, cast####################################
    
    URL = f"https://www.imdb.com/title/{title_id}/fullcredits?ref_=tt_cl_sm"
    resq = get_html(URL)
    soup = BeautifulSoup(resq, "html.parser")
    
    try:
        director_name = soup.select_one("#fullcredits_content > table:nth-child(2) > tbody > tr:nth-child(1) > td.name > a").text.strip()
        
    except AttributeError:
        try:
            director_name = soup.select_one("#fullcredits_content > table:nth-child(4) > tbody > tr > td.name > a").text.strip()
        except AttributeError:
            director_name = "NONE"
        
    director.append(director_name)
    
    
    ech_cast = []
    for i in range(2, 2 * cast_n +1,2):
        
        try:
            act = soup.select_one(f"#fullcredits_content > table.cast_list > tr:nth-child({i}) > td:nth-child(2) > a").text.strip()
            ech_cast.append(act)
        except AttributeError:
            if len(ech_cast) >= 1:
                break
            ech_cast.append("NONE")
            break
    
    cast.append(', '.join(ech_cast))
    
    return director, cast, cnty, title_id
    
        
# 결측값 채우기
def fill_null(x):
    
        
    keyword = x.title
    if pd.isna(x.director) or pd.isna(x.country) or pd.isna(x.cast):
        director, cast, country, id_ = fill_dir_cast_cnty(keyword) 
    else:
        return x
    
    if pd.isna(x.director):
        x.director = ''.join(director)
#         print(x.director)
    if pd.isna(x.country):
        x.country = ''.join(country)
        
        
    if pd.isna(x.cast):
        x.cast = ''.join(cast)
#         print(x.cast)

    x["title_ID"] = id_
    
    return x
        

In [416]:
fixed_df = df.apply(fill_null, axis = 1)

Title : Dick+Johnson+Is+Dead
Title : Blood+%26+Water
Title : Ganglands
Title : Jailbirds+New+Orleans
Title : Kota+Factory
Title : Midnight+Mass
Title : My+Little+Pony%3A+A+New+Generation
Title : Vendetta%3A+Truth%2C+Lies+and+The+Mafia
Title : Bangkok+Breaking
Title : Confessions+of+an+Invisible+Girl
Title : Crime+Stories%3A+India+Detectives
Title : Dear+White+People
Title : Europe%27s+Most+Dangerous+Man%3A+Otto+Skorzeny+in+Spain
Title : Falsa+identidad
Title : Intrusion
Title : Jaguar
Title : Monsters+Inside%3A+The+24+Faces+of+Billy+Milligan
Title : Resurrection%3A+Ertugrul
Title : Avvai+Shanmughi
Title : Go%21+Go%21+Cory+Carson%3A+Chrissy+Takes+the+Wheel
Title : Love+on+the+Spectrum
Title : Minsara+Kanavu
Title : Ankahi+Kahaniya
Title : Chicago+Party+Aunt
Title : Sex+Education
Title : Squid+Game
Title : Tayo+and+Little+Wizards
Title : The+Father+Who+Moves+Mountains
Title : The+Stronghold
Title : Angry+Birds
Title : Chhota+Bheem
Title : He-Man+and+the+Masters+of+the+Universe
Title : My

Title : Last+Summer
Title : Lee+Su-geun%3A+The+Sense+Coach
Title : Lethal+Love
Title : The+Cook+of+Castamar
Title : Virgin+River
Title : Elize+Matsunaga%3A+Once+Upon+a+Crime
Title : RESIDENT+EVIL%3A+Infinite+Darkness
Title : Awon+Boyz
Title : Cat+People
Title : Clash
Title : Dogs
Title : Major+Grom%3A+Plague+Doctor
Title : Mama+Drama
Title : The+War+Next-door
Title : Therapy
Title : Three+Thieves
Title : Ask+the+StoryBots
Title : Between
Title : Chelsea
Title : Code+Lyoko
Title : Colony
Title : Cuckoo
Title : Derry+Girls
Title : Girl+from+Nowhere
Title : Good+Morning+Call
Title : I+AM+A+KILLER
Title : I+Think+You+Should+Leave+with+Tim+Robinson
Title : Kim%27s+Convenience
Title : Larva
Title : Marcella
Title : Midnight+Diner%3A+Tokyo+Stories
Title : My+Little+Pony%3A+Friendship+Is+Magic
Title : Orange+Is+the+New+Black
Title : Puffin+Rock
Title : Richie+Rich
Title : Scream
Title : Shtisel
Title : Some+Assembly+Required
Title : Tales+by+Light
Title : Terrace+House%3A+Opening+New+Doors
Tit

Title : Jeffrey+Epstein%3A+Filthy+Rich
Title : Law+School
Title : Living+with+Yourself
Title : Signal
Title : Slasher
Title : Unnatural+Selection
Title : Why+Did+You+Kill+Me%3F
Title : Motu+Patlu+in+Hong+Kong%3A+Kung+Fu+Kings+3
Title : Motu+Patlu+in+the+City+of+Gold
Title : Motu+Patlu+Kung+Fu+Kings+4+The+Challenge+of+Kung+Fu+Brothers
Title : Motu+Patlu+the+Superheroes+–+Super+Villains+from+Mars
Title : My+Love%3A+Six+Stories+of+True+Love
Title : Shiva+VS+Autobots
Title : Shiva%3A+Journey+to+Plunotaria
Title : Shiva%3A+The+Secret+World+Of+Vedas+City
Title : The+Baker+and+the+Beauty
Title : Nicky%2C+Ricky%2C+Dicky+%26+Dawn
Title : Don%27t+be+the+First+one
Title : Heaven+Official%27s+Blessing
Title : The+Way+of+the+Househusband
Title : Dolly+Parton%3A+A+MusiCares+Tribute
Title : The+Big+Day
Title : The+Wedding+Coach
Title : This+Is+a+Robbery%3A+The+World%27s+Biggest+Art+Heist
Title : The+Last+Kids+on+Earth%3A+Happy+Apocalypse+to+You
Title : Coded+Bias
Title : Glimpses+of+a+Future
Title : 

Title : A+Lion+in+the+House
Title : DASH+%26+LILY
Title : Bakugan%3A+Armored+Alliance
Title : Bakugan%3A+Battle+Planet
Title : Country+Ever+After
Title : Wrong+Kind+of+Black
Title : Carmel%3A+Who+Killed+Maria+Marta%3F
Title : Paranormal
Title : A+Christmas+Catch
Title : Love+%26+Anarchy
Title : The+Beginning+of+Life
Title : Ya+no+estoy+aquí%3A+Una+conversación+entre+Guillermo+del+Toro+y+Alfonso+Cuarón
Title : Can+You+Hear+Me%3F
Title : 60+Days+In
Title : Dawson%27s+Creek
Title : Forged+in+Fire
Title : Leah+Remini%3A+Scientology+and+the+Aftermath
Title : LEGO+Jurassic+World%3A+Legend+of+Isla+Nublar
Title : LEGO+Jurassic+World%3A+Secret+Exhibit
Title : LEGO+Ninjago
Title : LEGO+Ninjago%3A+Masters+of+Spinjitzu
Title : LEGO%3A+CITY+Adventures
Title : The+Garfield+Show
Title : The+Good+Detective
Title : Voice
Title : Zumbo%27s+Just+Desserts
Title : Kongsuni+and+Friends
Title : Marvel%27s+Agents+of+S%2EH%2EI%2EE%2EL%2ED%2E
Title : Somebody+Feed+Phil
Title : Suburra%3A+Blood+on+Rome
Title : L

Title : Betaal
Title : Dynasty
Title : History+101
Title : Fifty+Year+Old+Teenager
Title : Lock+Your+Girls+In
Title : Mystic+Pop-up+Bar
Title : Raya+and+Sakina
Title : Sayed+the+Servant
Title : Especial+20+años+Fútbol+de+Primera
Title : The+Queen+and+the+Conqueror
Title : Sweet+Magnolias
Title : The+Big+Flower+Fight
Title : Patriot+Act+with+Hasan+Minhaj
Title : Boys+Over+Flowers
Title : Iris
Title : Avatar%3A+The+Last+Airbender
Title : Madam+Secretary
Title : Magic+for+Humans
Title : Parasyte%3A+The+Maxim
Title : She-Ra+and+the+Princesses+of+Power
Title : Strangers+from+Hell
Title : The+Wiggles
Title : White+Lines
Title : Match
Title : True%3A+Terrific+Tales
Title : Bordertown
Title : Trial+By+Media
Title : Dead+to+Me
Title : Restaurants+on+the+Edge
Title : The+Hollow
Title : Sí%2C+Mi+Amor
Title : Hangar+1%3A+The+UFO+Files
Title : Hoarders
Title : The+Universe%3A+Ancient+Mysteries+Solved
Title : WWII+in+HD
Title : Hollywood
Title : I+Am+Divine
Title : Masha+and+the+Bear
Title : Medici%

...

Title : Alejandro+Riaño%3A+Especial+de+stand+up
Title : Tundukkan+Playboy+Itu
Title : Episodes
Title : Eve%27s+Apple
Title : Fullmetal+Alchemist
Title : Lovesick
Title : Pokémon%3A+Indigo+League
Title : The+Frozen+Dead
Title : La+Mante
Title : Wadi
Title : Part-Time+Idol
Title : 72+Dangerous+Animals%3A+Latin+America
Title : The+Indian+Detective
Title : Hello%2C+My+Twenties%21
Title : Cheuk+Wan-Chi%3A+Two+Night+Stand
Title : Wormwood
Title : Halt+and+Catch+Fire
Title : Dawai+Asmara
Title : The+Secret
Title : All+Hail+King+Julien
Title : Dhia+Sofea
Title : DreamWorks+Home%3A+For+the+Holidays
Title : Kantaro%3A+The+Sweet+Tooth+Salaryman
Title : Padamu+Aku+Bersujud
Title : TURN%3A+Washington%27s+Spies
Title : Man+Down
Title : Broadchurch
Title : Cuba+and+the+Cameraman
Title : The+Many+Faces+of+Ito
Title : Trailer+Park+Boys%3A+Out+of+the+Park%3A+USA
Title : Bright%3A+The+Music+Videos
Title : Argon
Title : Godless
Title : Longmire
Title : Luna+Petunia
Title : Shot+in+the+Dark
Title : The+Irr

Title : Caught+on+Camera
Title : Real+Crime%3A+Supermarket+Heist+%28Tesco+Bomber%29
Title : Children+of+God
Title : Cocaine
Title : Edge+of+the+Universe
Title : La+Esclava+Blanca
Title : LEGO+Bionicle%3A+The+Journey+to+One
Title : Popples
Title : Marco+Polo
Title : Justin+Time+GO%21
Title : Hibana%3A+Spark
Title : 72+Cutest+Animals
Title : 72+Dangerous+Places+to+Live
Title : Team+Foxcatcher
Title : Patton+Oswalt%3A+Talking+for+Clapping
Title : Cyborg+009+VS+Devilman
Title : Frank+and+Cindy
Title : Justin+Time
Title : Terrace+House%3A+Boys+%26+Girls+in+the+City
Title : Netflix+Presents%3A+The+Characters
Title : Your+lie+in+April
Title : Cooked
Title : Care+Bears+%26+Cousins
Title : Turbo+FAST
Title : Masha%27s+Tales
Title : Chelsea+Does
Title : Sammy+%26+Co
Title : Atelier
Title : W%2F+Bob+%26+David
Title : Winter+on+Fire%3A+Ukraine%27s+Fight+for+Freedom
Title : Wakfu%3A+The+Quest+for+the+Six+Eliatrope+Dofus
Title : Wet+Hot+American+Summer
Title : H2O%3A+Mermaid+Adventures
Title : What+

Title : Gender+Revolution%3A+A+Journey+with+Katie+Couric
Title : Genius+of+the+Ancient+World
Title : Genius+of+the+Modern+World
Title : Gentlemen+and+Gangsters
Title : Get+Shorty
Title : Ghost+Town+Gold
Title : GLOW%3A+The+Story+of+the+Gorgeous+Ladies+of+Wrestling
Title : Glowing+Embers
Title : Golden+Time
Title : Goldie+%26+Bear
Title : Gomorrah
Title : Gossip+Girl
Title : Grand+Designs
Title : Grand+Hotel
Title : Great+Interior+Design+Challenge
Title : Great+Yellowstone+Thaw
Title : Gringo%3A+The+Dangerous+Life+of+John+McAfee
Title : Growing+Up+Coy
Title : Gun+Runners
Title : Gunslinger+Girl
Title : Ha+Unlimited
Title : Haapus
Title : Hannibal
Title : Happy+300+Days
Title : Happy+Valley
Title : Happyish
Title : Harlem+Globetrotters%3A+The+Team+That+Changed+the+World
Title : Harold+Shipman+-+Driven+to+Kill
Title : Harry+%26+Bunnie
Title : Hart+of+Dixie
Title : Hati+Perempuan
Title : Haunters%3A+The+Art+of+the+Scare
Title : Haven
Title : Heavy+Rescue%3A+401
Title : Helix
Title : Hell+o

Title : Power+Rangers+Dino+Charge
Title : Power+Rangers+Dino+Super+Charge
Title : Power+Rangers+Dino+Thunder
Title : Power+Rangers+in+Space
Title : Power+Rangers+Jungle+Fury
Title : Power+Rangers+Lightspeed+Rescue
Title : Power+Rangers+Lost+Galaxy
Title : Power+Rangers+Mystic+Force
Title : Power+Rangers+Ninja+Storm
Title : Power+Rangers+Operation+Overdrive
Title : Power+Rangers+RPM
Title : Power+Rangers+S%2EP%2ED%2E
Title : Power+Rangers+Samurai
Title : Power+Rangers+Samurai%3A+Christmas+Together%2C+Friends+Forever+%28Christmas+Special%29
Title : Power+Rangers+Samurai%3A+Clash+of+the+Red+Rangers
Title : Power+Rangers+Samurai%3A+Party+Monsters+%28Halloween+Special%29
Title : Power+Rangers+Super+Megaforce
Title : Power+Rangers+Super+Samurai
Title : Power+Rangers+Super+Samurai%3A+Stuck+on+Christmas
Title : Power+Rangers+Super+Samurai%3A+Trickster+Treat
Title : Power+Rangers+Time+Force
Title : Power+Rangers+Turbo
Title : Power+Rangers+Wild+Force
Title : Power+Rangers+Zeo
Title : Power+Rang

Title : Wild+Alaska
Title : Wild+Arabia
Title : Williams
Title : Winnie
Title : Winsanity
Title : Winter+Sun
Title : Women+Behind+Bars
Title : Woodstock
Title : World%27s+Busiest+Cities
Title : World%27s+Weirdest+Homes
Title : WWII%3A+Report+from+the+Aleutians
Title : Yeh+Meri+Family
Title : Yo-Kai+Watch
Title : Yoko
Title : Yoko+and+His+Friends
Title : YOM
Title : Yu-Gi-Oh%21+Arc-V
Title : Yunus+Emre
Title : Zak+Storm
Title : Zindagi+Gulzar+Hai
Title : Zombie+Dumb


In [432]:
fixed_df.drop("title_ID",axis=1, inplace=True)

In [434]:
fixed_df.head()

,cast,country,date_added,description,director,duration,listed_in,rating,release_year,show_id,title,type
0,"Michael Hilow, Dick Johnson, Chad Knorr, Vasth...",United States,"September 25, 2021","As her father nears the end of his life, filmm...",Kirsten Johnson,90 min,Documentaries,PG-13,2020,s1,Dick Johnson Is Dead,Movie
1,"Ama Qamata, Khosi Ngema, Gail Mabalane, Thaban...",South Africa,"September 24, 2021","After crossing paths at a party, a Cape Town t...",Nosipho Dumisa,2 Seasons,"International TV Shows, TV Dramas, TV Mysteries",TV-MA,2021,s2,Blood & Water,TV Show
2,"Sami Bouajila, Tracy Gotoas, Samuel Jouy, Nabi...",France,"September 24, 2021",To protect his family from a powerful drug lor...,Julien Leclercq,1 Season,"Crime TV Shows, International TV Shows, TV Act...",TV-MA,2021,s3,Ganglands,TV Show
3,NONE,United States,"September 24, 2021","Feuds, flirtations and toilet talk go down amo...",Jacquelyn C. Migdal,1 Season,"Docuseries, Reality TV",TV-MA,2021,s4,Jailbirds New Orleans,TV Show
4,"Mayur More, Jitendra Kumar, Ranjan Raj, Alam K...",India,"September 24, 2021",In a city of coaching centers known to train I...,Raghav Subbu,2 Seasons,"International TV Shows, Romantic TV Shows, TV ...",TV-MA,2021,s5,Kota Factory,TV Show


In [435]:
fixed_df.to_csv("fixed_netflix_df.csv", index = False)

In [439]:
fixed_df.iloc[8794,:]

cast            Nelly Karim, Hany Adel, Tarek Abdel Aziz, Ahme...
country                                             Egypt, France
date_added                                       October 11, 2018
description     Amid the tumult following Egyptian President M...
director                                             Mohamed Diab
duration                                                   98 min
listed_in        Dramas, Independent Movies, International Movies
rating                                                      TV-14
release_year                                                 2016
show_id                                                     s8795
title                                                      اشتباك
type                                                        Movie
Name: 8794, dtype: object